In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import os
import re
from urllib.request import urlopen

import pandas as pd
import numpy as np

In [2]:
from scrape import make_soup

In [18]:
soup1 = make_soup('https://minimalistbaker.com/recipes/')
posts = soup1.find_all('a', class_="entry-title-link")
links = [post.get("href") for post in posts]


In [10]:
def get_recipe_links(url):
    '''
    Gets all links to recipe post on one archival page.
    Returns list of strings containing the URLs
    '''
    soup = make_soup(url)
    posts = soup.find_all('a', class_="entry-title-link")
    links = [post.get("href") for post in posts]
    return links

Psuedocode for getting all recipe posts

- for every page
    - get the links for the posts on the page 
    - for every post on a page
        - get the text body of that post
        - save it into an array / dataframe
    

In [15]:
def grab_body(url):
    '''
    INPUT: blog post url
    OUTPUT: list of sentence strings
    '''
    soup = make_soup(url)
    art = soup.article
    recipe = art.find('div', class_="wprm-recipe-container")
    
    recipe_start = str(recipe)[:30]
    idx = str(art).index(recipe_start)
    data = str(art)[:idx]
    data = re.sub(r'<.*?>', '', data)
    data = re.sub('\xa0', '', data)
    data = re.sub('\n', ' ', data)
    return data, soup.title.text


In [25]:
def get_bodies_on_page(url):
    df = pd.DataFrame(columns=['title', 'body', 'url'])
    links = get_recipe_links(url)
    for link in links:
        body, title = grab_body(link)
        A = pd.DataFrame([[title, body, url]], columns=['title', 'body', 'url'])
        df = pd.concat([df,A], axis=0)
    return df

In [23]:
def scrape_mb(url):
    data = pd.DataFrame(columns=['title','body','url'])
    data = pd.concat([data, get_bodies_on_page(url)], axis=0)
    for n in range(2, 61):
        url = 'https://minimalistbaker.com/recipes/{}/'.format(i)
        df = get_bodies_on_page(url)
        data = pd.concat([data, df], axis=0)
    

[<a href="https://minimalistbaker.com/" title="Minimalist Baker"><img alt="Minimalist Baker" height="103px" src="https://minimalistbaker.com/wp-content/themes/baker/images/logo.png" title="Minimalist Baker" width="300px"/></a>,
 <a href="https://minimalistbaker.com/about/" itemprop="url"><span itemprop="name">about</span></a>,
 <a href="https://minimalistbaker.com/recipe-index/" itemprop="url"><span itemprop="name">recipes</span></a>,
 <a href="https://minimalistbaker.com/product-category/cookbooks/" itemprop="url"><span itemprop="name">Cookbooks</span></a>,
 <a href="https://minimalistbaker.com/everyday-cooking/" itemprop="url" target="_blank"><span itemprop="name">Everyday Cooking (print)</span></a>,
 <a href="https://minimalistbaker.com/shop/" itemprop="url"><span itemprop="name">Shop</span></a>,
 <a href="https://minimalistbaker.com/resources/" itemprop="url"><span itemprop="name">blogger resources</span></a>,
 <a href="https://minimalistbaker.com/product-category/products/" itempr